In [13]:
#calculo
import numpy as np
import pandas as pd

#grafico
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
def error_print(text):
    print("*"*50)
    print("ERROR:", text)
    print("*"*50)

In [36]:
class Analizer:
    def __init__(self, ruta='', sep=','):
        func_read = pd.read_csv if ruta.lower().endswith('.csv') else None
        if not func_read:
            error_print('archivo .csv no detectado')
            self.data = None
            return
            
        try:
            self.data = pd.read_csv(ruta, encoding= "ISO-8859-1", sep=sep)
        except:
            error_print('verifique ubicacion de archvio')
            self.data = None
            return
        self.test_data = None
        self.col_target = ''
        self.target = None
        self.columns = list(self.data.columns)
        self.columns_set = set(self.columns)

    def null_data(func):
        def proccess(self, *args, **kwargs):
            if self.data is None:
                error_print('data no extraida')
                return
            return func(self, *args, **kwargs)
        return proccess
    
    @null_data
    def head(self, num=5):
        return self.data.head(num)
        
    @null_data
    def set_test_data(self, datus):
        self.test_data = datus.data if isinstance(datus, self.__class__) else (
            datus if isinstance(datus, pd) else None
        )
        return self.test_data.head() if self.test_data is not None else 'Test no asignado'
    
    @null_data
    def set_target(self, col_name):
        if col_name not in self.columns:
            error_print('Columna no identificada')
            return
        
        self.col_target = col_name
        self.target = self.data[col_name]
        self.data = self.data.drop([col_name], axis=1)
        return self.head()
        
    @property
    def columns_null(self):
        return [
            col for col in self.columns_set if self.data[col].isnull().any()
        ]   
    
    def cols_for_types(self, cols=[]):
        if not self.validate_list(cols):
            error_print("Campo requerido como lista invalido")
            return
        
        if not set(cols).issubset(self.columns_set):
            error_print("Columnas invalidas detectadas")
            return 
        
        print('validado')
        return {
            str(k).lower(): [col for col in list(v) if col in cols]
            for k, v in 
            self.data.columns.to_series().groupby(self.data.dtypes).groups.items()
        }

    @property
    def types(self):
        return [str(tipo) for tipo in set(self.data.dtypes)]
    
    
    @null_data
    def counts(self):
        for col in self.data.columns.values:
            print("-"*40)
            print(self.data[col].value_counts(normalize = True, dropna = False))

    
    @null_data
    def null_verificator(self):        
        if self.data.isnull().any().any():
            view_info = pd.DataFrame(
                pd.concat(
                    [self.data.isnull().any(), 
                     self.data.isnull().sum(),
                     self.data.dtypes], 
                    axis=1)
            )
            view_info.columns = ['Nulos', 'Cantidad', 'Tipo Col']
            return view_info
        else:
            return "DATA LIMPIA DE NULOS"
        
    @null_data
    def describe(self):
        return self.data.describe(include='all')
    
    @null_data
    def percentil_verificator(
        self, cols=[], list_percentil=[0,1,3,5,10,20,30,50,60,70,80,90,95,97,99,100]
    ):
        if not isinstance(cols, list):
            print("'cols' debe contener una lista ")
            
        datus = self.data.copy()
        for col in cols:
            plt.plot(
                list_percentil,
                np.nanpercentile(datus[col], list_percentil),
                label=col
            )
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.show()
        df_datus = pd.concat(
            [ 
                pd.Series(np.nanpercentile(datus[col], list_percentil),
                          index=list_percentil) 
                for col in cols
            ],
            axis=1
        )
        df_datus.columns = cols
        return df_datus.T
    
    @null_data
    def cut_col_percentil(self, cols=[], percentile_bigger=95, percentile_smaller=0, 
                          val_bigger=[], val_smaller=[], replicate_in_test=False):

        if not self.validate_list(cols, val_bigger, val_smaller):
            error_print("Campo requerido como lista invalido")
            return
        
        if not self.validate_percentil(percentile_bigger, percentile_smaller):
            error_print("Campo requerido como entero invalido")
            return
        
        if not set(cols).issubset(self.columns_set):
            error_print("Columnas invalidas detectadas")
            return 
        
        if val_bigger and len(val_bigger) != len(cols):
            error_print("la cantidad de valores debe corresponder a cada columna")
            return

        if val_smaller and len(val_smaller) != len(cols):
            error_print("la cantidad de valores debe corresponder a cada columna")
            return 
        
        if replicate_in_test and self.test_data is None:
            error_print("Data test no especificada")
            return            

        for i, col in enumerate(cols):
            copy_col = self.data[col].copy()
            
            percent_cut = val_bigger[i] if val_bigger else copy_col.quantile(percentile_bigger/100)
            self.data.loc[self.data[col] > percent_cut, col] = percent_cut
            if replicate_in_test:
                self.test_data.loc[self.test_data[col] > percent_cut, col] = percent_cut
                
            if val_smaller or percentile_smaller:
                percent_cut = val_smaller[i] if val_smaller else copy_col.quantile(percentile_smaller/100)
                self.data.loc[self.data[col] < percent_cut, col] = percent_cut
                if replicate_in_test:
                    self.test_data.loc[self.test_data[col] < percent_cut, col] = percent_cut
        
        del copy_col
        if not replicate_in_test:
            return self.data[cols].describe(include='all')
        else:
            return pd.DataFrame(
                pd.concat(
                    [self.data[cols].describe(include='all'), 
                     self.test_data[cols].describe(include='all')], 
                    axis=1)
            )
    
    def validate_percentil(self, *args):
        return all([isinstance(_, int) and _ in range(0,100) for _ in args])
        
    
    def validate_list(self, *args):
        return all([isinstance(_, list) for _ in args])

    
    @null_data
    def add_col_dates(self, col, format_match="%d-%b-%y", month=True, day=True, month_day=True,
                      weekday=True, replace_str=False, format_str_replace='%Y/%m/%d'):
        
        self.data['date'] = pd.to_datetime(self.data[col], format=format_match)
        self.data = self.data.drop([col], axis=1)
        
        if month:
            self.data['month'] = pd.to_numeric(self.data['date'].dt.strftime('%m'), errors='coerce')
            self.data['month'].fillna(-99)
        if day:
            self.data['day'] = pd.to_numeric(self.data['date'].dt.strftime('%d'), errors='coerce')
            self.data['day'].fillna(-99)
        if month_day:
            self.data['month_day'] = pd.to_numeric(self.data['date'].dt.strftime('%m%d'), errors='coerce')
            self.data['month_day'].fillna(-99)
        if weekday:
            self.data['weekday'] = pd.to_numeric(self.data['date'].dt.strftime('%w'), errors='coerce')
            self.data['weekday'].fillna(-99)
        if replace_str:
            self.data['date'] = self.data['date'].dt.strftime(format_str_replace)
            
        return(self.data.head(10))
            

        

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,524931,6535,07-Feb-04,0,0,0,0,0
3,494493,6570,20-May-05,0,0,0,0,0
4,523001,6562,22-May-05,0,0,0,0,0


,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,523729,6534,29-Apr-06,0,0,0,0,0
1,521364,6529,28-Feb-06,15,1,0,0,7
2,506153,6534,25-Mar-06,0,0,0,0,0
3,514205,6571,15-Sep-05,0,0,0,0,0
4,496155,6541,18-Nov-05,0,0,0,0,0


In [39]:
train_analisis.cols_for_types(cols=train_analisis.columns_null)

validado


{'int64': [], 'object': ['Registration_Date']}

,Var1,Var2,Var1,Var2
count,58532.000000,58532.0,16746.000000,16746.0
mean,0.239664,0.0,0.172220,0.0
std,0.741550,0.0,0.633555,0.0
min,0.000000,0.0,0.000000,0.0
25%,0.000000,0.0,0.000000,0.0
50%,0.000000,0.0,0.000000,0.0
75%,0.000000,0.0,0.000000,0.0
max,3.000000,0.0,3.000000,0.0
